# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: ...

In [19]:
#!pip install hypothesis

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обеих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

In [1]:
import numpy as np
from typing import Optional, Union, Tuple

In [2]:
def euclidean_distance(x: np.array, y: np.array) -> float:
    """
    Calculate euclidean distance between points x and y
    Args:
        x, y: two points in Euclidean n-space
    Returns:
        Length of the line segment connecting given points
    """
    return sum((x - y) ** 2) ** 0.5
    pass


def euclidean_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate euclidean similarity between points x and y
    Args:
        x, y: two points in Euclidean n-space
    Returns:
        Similarity between points x and y
    """
    return 1 / (1 + euclidean_distance(x, y))
    pass


def pearson_similarity(x: np.array, y: np.array) -> float:
    """
    Calculate a Pearson correlation coefficient given 1-D data arrays x and y
    Args:
        x, y: two points in n-space
    Returns:
        Pearson correlation between x and y
    """
    return sum((x - x.mean()) * (y - y.mean())) / ((sum((x - x.mean()) ** 2) * sum((y - y.mean()) ** 2)) ** 0.5)
    pass

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [3]:
import numpy as np
from utils import euclidean_similarity, pearson_similarity

In [4]:
def del_zeros(x: np.array, y: np.array):
    tmp_1 = list(map(lambda el: el != 0, x))
    tmp_2 = list(map(lambda el: el != 0, y))
    mask = np.array([tmp_1[i] and tmp_2[i] for i in range(len(tmp_1))])
    return x[mask], y[mask]

def euclidean_similarity_edit(x: np.array, y: np.array) -> float:
    x, y = del_zeros(x, y)
    if len(x) == 0:
        return -np.inf
    return 1 / (1 + euclidean_distance(x, y))
    pass


def pearson_similarity_edit(x: np.array, y: np.array) -> float:
    x, y = del_zeros(x, y)
    if len(x) == 0:
        return -np.inf
    return sum((x - x.mean()) * (y - y.mean())) / ((sum((x - x.mean()) ** 2) * sum((y - y.mean()) ** 2)) ** 0.5)
    pass


In [5]:
a = np.array([0,0,0])
b = np.array([0,0,0])

print(euclidean_similarity_edit(a, b))
print(pearson_similarity_edit(a, b))


print(euclidean_similarity_edit(a, b))
print(pearson_similarity_edit(a, b))

-inf
-inf
-inf
-inf


In [24]:
!pytest test.py

============================= test session starts =============================
platform win32 -- Python 3.8.5, pytest-6.1.1, py-1.9.0, pluggy-0.13.1
rootdir: C:\Users\Александр\Desktop\тинькофф\10 lecture
plugins: hypothesis-6.12.0
collected 11 items

test.py ...........                                                      [100%]

============================== warnings summary ===============================
D:\anaconda3\lib\site-packages\pyreadline\py3k_compat.py:8
  D:\anaconda3\lib\site-packages\pyreadline\py3k_compat.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
    return isinstance(x, collections.Callable)

test.py::test_pearson_similarity
  C:\Users\Александр\Desktop\тинькофф\10 lecture\utils.py:16: RuntimeWarning: invalid value encountered in double_scalars
    return sum((x - x.mean()) * (y - y.mean())) / ((sum((x - x.mean()) ** 2) * sum((y - y.mean())

In [ ]:
#benhamner/Metrics git

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [14]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.metric_ = metric
        self.n_recommendations_ = n_recommendations
        self.alpha_ = alpha
        pass
    
    def sim(self, x: np.array, y: np.array) -> float:
        if self.metric_ == 'euclidean':
            return euclidean_similarity_edit(x, y)
        elif self.metric_ == 'pearson':
            return pearson_similarity_edit(x, y)

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        N = len(X)
        self.data = np.array([np.array([1 - sim(X[i], X[j]) if i != j else np.inf for j in range(N)]) for i in range(N)])
        pass

    def _find_closest_users(self, user_id: int, n_closest_users: int):
        self.data = self.data[np.argsort(self.data[:, user_id])]
        tmp_data = np.array([[i, d] for i, d in enumerate(self.data[:, user_id])])
        tmp_arr = tmp_arr[tmp_arr[:, 1].argsort()]
        return tmp_data[:n_closest_users, :]
        pass

    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        tmp_rec_id = _find_closest_users(user_id, n_recommendations)
        rec_id = []
        for i in range(user_id):
            if 1 - tmp_rec_id[i, 1] > self.alpha:
                rec_id.append(int(tmp_rec_id[i, 0]))
        return np.array(rec_id)
        pass

In [55]:
import numpy as np

arr = np.array([[-0.30565392, -0.96605562],
                [ 0.85331367, -2.62963495],
                [ 0.87839643, -0.28283675],
                [ 0.72676698,  0.93213482],
                [-0.52007354,  0.27752806],
                [-0.08701666,  0.22764316],
                [-1.78897817,  0.50737573],
                [ 0.62260038, -1.96012161],
                [-1.98231706,  0.36523876],
                [-1.07587382, -2.3022289 ]])

tmp_arr = np.array([[i, p] for i, p in enumerate(arr[:, 1])])

tmp_arr = tmp_arr[tmp_arr[:, 1].argsort()]

tmp_arr[:4,0]

array([1., 9., 7., 0.])

In [18]:
a = UserBasedRecommendation()

a.metric_

'euclidean'

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [33]:
#Например:

a = np.array([i for i in range(10)])
b = np.array([i for i in range(9, -1, -1)])

print(euclidean_similarity(a, b), pearson_similarity(a, b))

#наборы одинаковы, с точностью до перестановки, что прекрасно определяет коэфф. Пирсона
#в отличие от Евклидового рассотяния

a = np.array([1,2,3,4])
b = np.array([1,3,2,4])

print(euclidean_similarity(a, b), pearson_similarity(a, b))

#оба примера показываеют, что Пирсон спасает, если нам необходимо из двух наборово векторов определить схожесть
#с учетом того, что порядок координат не важен. Если сравнивать категорично (в Евклидовом смысле), то мы получим 
#существенные отклонения

0.05217599429965031 -1.0
0.4142135623730951 0.8


<b>Объяснение:</b> Для ясности пронумеруем товары с 0. Видно, что третьему пользователю так же как и второму больше понравился товар 1 и чуть меньше понравился товар 2, тогда как первому пользователю оба эти товары не сильно понравились, причем товар 1 ему понравился меньше, чем товар 2, то есть наблюдается обратная завизимость. Но все эти факторы никак не учитываются при использовании евклидовой метрики, поэтому в первом случае алгоритм посчитал, что первый и третий пользователь похожи только по тому, что они ставят оценки примерно в одном и том отрезке: {0, 1}, что является довольно странным предположением. А алгоритм, использующий коэффициент корреляции Пирсона учитывает эти факторы, поэтому находит третьему пользователю соседа со схожими интересами в виде второго пользователя. Отсюда получаем разные рекомендации.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

In [20]:
def apk(actual: np.array, predicted: np.array, k: int = 10) -> float:
    """
    Compute the average precision at k
    Args:
        actual: a list of elements that are to be predicted (order doesn't matter)
        predicted: a list of predicted elements (order does matter)
        k: the maximum number of predicted elements
    Returns:
        The average precision at k over the input lists
    """
    if k < len(predicted):
        predicted = predicted[:k]

    score = 0
    num_hits = 0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1
            score += num_hits / (i + 1)

    if not num_hits:
        return 0

    return score / min(len(actual), k)
    pass


def mapk(actual: np.array, predicted: np.array, k: int = 10) -> float:
    """
    Compute the mean average precision at k
    Args:
        actual: a list of lists of elements that are to be predicted
        predicted: a list of lists of predicted elements
        k: the maximum number of predicted elements
    Returns:
        The mean average precision at k over the input lists
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])
    pass

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

In [ ]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

In [ ]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

Удалим для наиболее активных пользователей 5 оценок

In [ ]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [ ]:
from utils import mapk

In [ ]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
pass

Как можно улучшить работу модели?

<b>Ответ:</b>...